The data file contains data on bank transactions of a company in FY2018:

- Transaction id;
- Purchase order id (each transaction belongs to a purchase order, i.e., each purchase may involve multiple bank transactions);
- The net worth of the transaction in Euros;
- The date of the transaction;

As part of an internal audit, the database must be analysed to establish if it contains any erroneous or fraudulent transactions. As it is impossible to manually verify each purchase order, the auditor would like to use a machine learning model that will automatically label suspicious purchase orders, so they can later be checked by hand. The first step in constructing the model is to extract predictive features from the data.

Your goal is to create predictive features for each Purchase Order that will capture:
- The number of transactions in the Purchase Order: too many transactions within one order may be suspicious.
- The average size of the transactions of the Purchase Order: too small or too large transactions are likely to be fraudulent.
- The spread of the money amounts of the transactions: the more varied are the amounts of the transactions, the more suspicious is the purchase order.
- The spread of the transactions in time: the larger the spread, the more suspicious is the purchase order.
- If the Purchase Order contains transactions outside of the financial year, i.e. before 06-04-2018 or after 06-04-2019.

The result must be a dataframe containing Purchase Order id and the five columns representing the five predictive features.

The source of the data is https://icpmconference.org/2019/icpm-2019/contests-challenges/bpi-challenge-2019/

In [3]:
import pandas as pd
from datetime import datetime

In [4]:
# load the data
df = pd.read_csv("/content/fraud_detection.csv")

In [6]:
# check columns and their datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1595923 entries, 0 to 1595922
Data columns (total 4 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   transaction_Id   1595923 non-null  int64  
 1   PO_Id            1595923 non-null  int64  
 2   net worth (EUR)  1595923 non-null  float64
 3   timestamp        1595923 non-null  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 48.7+ MB


In [8]:
# inspect the first several rows
df.head()

,transaction_Id,PO_Id,net worth (EUR),timestamp
0,65781719105536,4507004931,68.0,26-01-1948 23:59:00
1,65777424138241,4507004931,325.0,26-01-1948 23:59:00
2,65777424138240,4507004931,325.0,26-01-1948 23:59:00
3,65794604007424,4507004931,102.0,26-01-1948 23:59:00
4,65794604007425,4507004931,102.0,26-01-1948 23:59:00


In [10]:
# convert timestamp to datetime
df["timestamp"] = pd.to_datetime(df["timestamp"], format="%d-%m-%Y %H:%M:%S")

In [12]:
def days_diff(group_col):
    diff = group_col.max() - group_col.min()
    return diff.days

def date_outside_2018(group_col):
    if group_col.max() > datetime(2019, 4, 6):
        return False
    elif group_col.min() < datetime(2018, 4, 6):
        return False
    else:
        return True

In [15]:
# extract features
df.groupby("PO_Id").size()

,0
PO_Id,
2000000000,12
2000000001,15
2000000002,18
2000000003,36
2000000004,12
...,...
4508076344,5
4508076345,29
4508076346,26


In [20]:
df.groupby("PO_Id").mean("net worth (EUR)")

,transaction_Id,net worth (EUR)
PO_Id,,
2000000000,5.500000e+00,298.000000
2000000001,4.294967e+09,557.000000
2000000002,8.589935e+09,76210.000000
2000000003,1.717987e+10,36.333333
2000000004,2.576980e+10,166.000000
...,...,...
4508076344,1.081026e+15,574.000000
4508076345,1.081057e+15,6383.000000
4508076346,1.081108e+15,2714.000000
